<a href="https://colab.research.google.com/github/rdeepu13/SSTA/blob/main/SSTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import numpy as np
from scipy import stats
from scipy.linalg import eigh
from scipy.stats import norm
from scipy.special import erf
import networkx as nx
import matplotlib.pyplot as plt



```
Computes the mean and standard deviation of the sum of two Gaussian (normal) distributions.
Args:
        mean1: Mean of the first Gaussian distribution.
        std_dev1: Standard deviation of the first Gaussian distribution.
        mean2: Mean of the second Gaussian distribution.
        std_dev2: Standard deviation of the second Gaussian distribution.

Returns:
        A tuple containing the mean and standard deviation of the resulting Gaussian distribution.

```



In [2]:
def sum_gaussians(dist1, dist2):
    """Sums two Gaussian distributions."""
    result_mean = dist1.mean() + dist2.mean()
    result_std_dev = math.sqrt(dist1.std()**2 + dist2.std()**2)
    return norm(result_mean, result_std_dev)

# Example Gaussian distributions
dist1 = norm(5, 2)  # Mean = 3, Standard Deviation = 1
dist2 = norm(10, 3)  # Mean = 5, Standard Deviation = 2

# Sum the Gaussian distributions
sum_distribution = sum_gaussians(dist1, dist2)

# Print the result
print("Sum of Gaussian Distributions:")
print("Mean:", sum_distribution.mean())
print("Standard Deviation:", sum_distribution.std())

Sum of Gaussian Distributions:
Mean: 15.0
Standard Deviation: 3.605551275463989


```
Approximate the mean and standard deviation of the maximum of two Gaussian distributions.

    Args:
        mean_di (float): Mean of the first Gaussian distribution.
        stddev_di (float): Standard deviation of the first Gaussian distribution.
        mean_dj (float): Mean of the second Gaussian distribution.
        stddev_dj (float): Standard deviation of the second Gaussian distribution.
        cov_dij (float): Covariance between the two Gaussian distributions.

    Returns:
        tuple: Approximated mean and standard deviation of the maximum.
```

In [3]:
import math
from scipy.stats import norm

def calculate_alpha(mean_di, stddev_di, mean_dj, stddev_dj, cov_dij):
    alpha = math.sqrt(stddev_di**2 + stddev_dj**2 - 2 * stddev_di * stddev_dj * cov_dij)
    return alpha

def approximate_dmax(mean_di, stddev_di, mean_dj, stddev_dj, cov_dij):
    # Check if standard deviations are zero (to avoid division by zero)
    if stddev_di == 0:
        return norm(mean_dj, stddev_dj)  # Return the distribution with non-zero variance
    if stddev_dj == 0:
        return norm(mean_di, stddev_di)
    alpha = calculate_alpha(mean_di, stddev_di, mean_dj, stddev_dj, cov_dij)

    # Handle cases where alpha is very close to zero (to avoid numerical instability)
    if abs(alpha) < 1e-10:
        alpha = 1e-10  # Set a small value to avoid division by zero
    # Calculate beta
    beta = (mean_di - mean_dj) / alpha
    # Calculate mean of d_max
    mu_t = mean_di * norm.cdf(beta) + mean_dj * norm.cdf(-beta) + alpha * norm.pdf(beta)
    # Calculate variance of d_max
    variance_t = (mean_di**2 + stddev_di**2) * norm.cdf(beta) + (mean_dj**2 + stddev_dj**2) * norm.cdf(-beta) + (mean_di + mean_dj) * alpha * norm.pdf(beta) - mu_t**2
    # Ensure variance is non-negative
    variance_t = max(variance_t, 0)
    stddev_t = math.sqrt(variance_t)
    return norm(mu_t, stddev_t)

# Example usage d_max:
mean_di = 10
stddev_di = 2
mean_dj = 12
stddev_dj = 3
cov_dij = 0.5
# Calculate the approximate maximum distribution
max_dist = approximate_dmax(mean_di, stddev_di, mean_dj, stddev_dj, cov_dij)
# Print mean and standard deviation separately
print("Approximated mean of d_max:", max_dist.mean())
print("Approximated standard deviation of d_max:", max_dist.std())

Approximated mean of d_max: 12.343494033975771
Approximated standard deviation of d_max: 2.6590965168171907


```
 calculate the total delay of a path through the circuit by summing up the delays of each gate and wire it passes through, as we have confirmed earlier. If a gate delay for a node or a wire delay for an edge isn't present, it will show a warning and assume 0 delay.
 ```

In [4]:
wire_delay_params = {
    ('SOURCE', '1'): norm(0, 0),  # Virtual source to input
    ('SOURCE', '2'): norm(0, 0),
    ('SOURCE', '3'): norm(0, 0),
    ('SOURCE', '6'): norm(0, 0),
    ('SOURCE', '7'): norm(0, 0),
    ('1', '10'): norm(0.5, 0.05),
    ('3', '10'): norm(1.0, 0.1),
    ('3', '11'): norm(0.6, 0.08),
    ('6', '11'): norm(0.7, 0.09),
    ('2', '16'): norm(0.4, 0.04),
    ('11', '16'): norm(0.8, 0.1),
    ('11', '19'): norm(0.55, 0.06),
    ('7', '19'): norm(0.9, 0.12),
    ('10', '22'): norm(0.3, 0.03),
    ('16', '22'): norm(1.1, 0.15),
    ('16', '23'): norm(0.7, 0.1),
    ('19', '23'): norm(1.2, 0.18),
    ('22', 'OUTPUT(22)'): norm(0.01, 0.001), # Output to virtual sink,
    ('23', 'OUTPUT(23)'): norm(0.01, 0.001)
}

# --- Enhanced Graph Structure ---
graph_data = {
    '1': {'type': 'INPUT'},
    '2': {'type': 'INPUT'},
    '3': {'type': 'INPUT'},
    '6': {'type': 'INPUT'},
    '7': {'type': 'INPUT'},
    '10': {'type': 'NAND', 'inputs': ['1', '3'], 'gate_delay': norm(1.0, 0.1)},
    '11': {'type': 'NAND', 'inputs': ['3', '6'], 'gate_delay': norm(1.2, 0.15)},
    '16': {'type': 'NAND', 'inputs': ['2', '11'], 'gate_delay': norm(0.9, 0.09)},
    '19': {'type': 'NAND', 'inputs': ['11', '7'], 'gate_delay': norm(1.1, 0.12)},
    '22': {'type': 'NAND', 'inputs': ['10', '16'], 'gate_delay': norm(0.8, 0.08)},
    '23': {'type': 'NAND', 'inputs': ['16', '19'], 'gate_delay': norm(1.3, 0.16)},
    'OUTPUT(22)': {'type': 'OUTPUT', 'source': '22'},
    'OUTPUT(23)': {'type': 'OUTPUT', 'source': '23'}
}


In [5]:
# --- Graph Creation ---
G = nx.DiGraph()
virtual_source = 'SOURCE'
virtual_sink = 'SINK'

# Add all nodes to the graph (including virtual and output nodes)
for node, data in graph_data.items():
    G.add_node(node, **data)  # Add all node attributes directly

# Add edges from 'wire_delay_params'
for (u, v), delay_dist in wire_delay_params.items():
    G.add_edge(u, v, delay=delay_dist)  # Add edges and their delays



In [6]:
import networkx as nx
from scipy.stats import norm

def calculate_es_ls(G):
    # Initialize ES for all nodes with a normal distribution
    for node in G.nodes:
        G.nodes[node]['es'] = norm(0.0001, 0.00001)  # Start with a small ES distribution

    # Perform forward pass to calculate ES
    for node in nx.topological_sort(G):
        max_es_mean = 0.0001
        max_es_std = 0.00001

        for predecessor in G.predecessors(node):
            delay_dist = G[predecessor][node]['delay']
            predecessor_es_sample = G.nodes[predecessor]['es'].rvs()
            x = predecessor_es_sample + delay_dist.rvs()  # Sampled sum of predecessor's ES and delay

            # Update max ES mean and std
            max_es_mean = max(max_es_mean, x)  # Update mean
            max_es_std = (max_es_std**2 + delay_dist.std()**2)**0.5  # Update std

        # Update ES for the current node with the calculated max ES
        G.nodes[node]['es'] = norm(max_es_mean, max_es_std)

# Example usage:
G = nx.DiGraph()

# Add nodes and edges to the graph
graph_data = {
    '1': {'type': 'INPUT'},
    '2': {'type': 'INPUT'},
    '3': {'type': 'INPUT'},
    '6': {'type': 'INPUT'},
    '7': {'type': 'INPUT'},
    '10': {'type': 'NAND'},
    '11': {'type': 'NAND'},
    '16': {'type': 'NAND'},
    '19': {'type': 'NAND'},
    '22': {'type': 'NAND'},
    '23': {'type': 'NAND'},
    'OUTPUT(22)': {'type': 'OUTPUT'},
    'OUTPUT(23)': {'type': 'OUTPUT'}
}

wire_delay_params = {
    ('SOURCE', '1'): norm(0.5, 0.05),
    ('SOURCE', '2'): norm(0.4, 0.04),
    ('SOURCE', '3'): norm(0.6, 0.06),
    ('SOURCE', '6'): norm(0.7, 0.07),
    ('SOURCE', '7'): norm(0.8, 0.08),
    ('1', '10'): norm(0.5, 0.05),
    ('3', '10'): norm(1.0, 0.1),
    ('3', '11'): norm(0.6, 0.08),
    ('6', '11'): norm(0.7, 0.09),
    ('2', '16'): norm(0.4, 0.04),
    ('11', '16'): norm(0.8, 0.1),
    ('11', '19'): norm(0.55, 0.06),
    ('7', '19'): norm(0.9, 0.12),
    ('10', '22'): norm(0.3, 0.03),
    ('16', '22'): norm(1.1, 0.15),
    ('16', '23'): norm(0.7, 0.1),
    ('19', '23'): norm(1.2, 0.18),
    ('22', 'OUTPUT(22)'): norm(0.01, 0.001),
    ('23', 'OUTPUT(23)'): norm(0.01, 0.001)
}

# Add nodes and edges to the graph based on graph_data and wire_delay_params
for node, data in graph_data.items():
    G.add_node(node, **data)

for (u, v), delay_dist in wire_delay_params.items():
    G.add_edge(u, v, delay=delay_dist)

# Calculate ES for each node in the graph
calculate_es_ls(G)

# Print ES for each node
for node in G.nodes:
    es_distribution = G.nodes[node]['es']
    print(f"Node {node}: ES = {es_distribution.mean()}, Std = {es_distribution.std()}")


Node 1: ES = 0.46751470905221687, Std = 0.050000000999999995
Node 2: ES = 0.3989792933943225, Std = 0.04000000124999998
Node 3: ES = 0.5795024306338146, Std = 0.060000000833333324
Node 6: ES = 0.7173045721375872, Std = 0.0700000007142857
Node 7: ES = 0.7259947505718953, Std = 0.080000000625
Node 10: ES = 1.5795927333193573, Std = 0.1118033993222031
Node 11: ES = 1.381044157178528, Std = 0.12041594620315035
Node 16: ES = 2.1811144998292527, Std = 0.10770329660692844
Node 19: ES = 2.0516812154061377, Std = 0.1341640790226654
Node 22: ES = 3.5258764958031747, Std = 0.15297058573464375
Node 23: ES = 3.231079116624959, Std = 0.20591260306256148
Node OUTPUT(22): ES = 3.3918286048818738, Std = 0.0010000499987500625
Node OUTPUT(23): ES = 3.064769930967477, Std = 0.0010000499987500625
Node SOURCE: ES = 0.0001, Std = 1e-05
